In [ ]:
import os
from langgraph.graph import StateGraph, START, END
# from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
# import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [34]:
load_dotenv()

True

In [35]:
# Get the API key from the environment variable loaded by dotenv
API_KEY = os.getenv("GEMINI_API_KEY")

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Or "gemini-pro"
    temperature=0.7,          # Controls creativity (0.0 to 1.0)
    google_api_key=API_KEY    # This is how LangChain gets your API key for Gemini
)


In [37]:
# Define the state
class LLMState(TypedDict):
    user_input: str
    llm_response: str

In [38]:
def llm_qa(state: LLMState) -> LLMState:
    
    user_input = state['user_input']

    # Prompt the LLM with the user input
    prompt = f"Anser the following question: {user_input}"
    llm_response = model.invoke(prompt).content
    state['llm_response'] = llm_response

    return state

In [39]:
# Define the state graph
graph = StateGraph(LLMState)

# Add nodes
graph.add_node("llm_qa", llm_qa)
# Add edges
graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

# Compile the graph
workflow = graph.compile()


In [41]:
# Execute the workflow
initial_state = LLMState(user_input="What is the capital of Pakistan?")
final_state = workflow.invoke(initial_state)
print(final_state['llm_response'])

Islamabad
